In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
warnings.filterwarnings("ignore", category =UserWarning,)

import gc
import pickle
from collections import OrderedDict
from pyproj import Transformer
import numpy as np
import pandas as pd
import iris
import glob
import sys
import os
import cartopy.crs as ccrs
import itertools
from scipy import spatial
import numpy.ma as ma
import tilemapbase
from math import cos, radians
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import time

from Identify_Events_Functions import *
from Prepare_Data_Functions import *

pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings("ignore", category=UserWarning)

yrs_range= '2002_2020'
em = 'bc006'

# Get Tb0 values at each gauge
tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# Read in a sample cube for finding the location of gauge in grid
sample_cube = iris.load(f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr200601.nc')[0][1,:,:]

for yr in range(2010,2021):
    print(yr)
    ######################################################
    ### Get all the data for one year, into one cube
    # (if it already exists in a pickle file, then load it from there)
    ######################################################
    general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{yr}*'
    pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/UKCP18_30mins_{em}/WholeYear/cube_{yr}.pkl"

    if os.path.exists(pickle_file_filepath):
        print("Pickle file exists, so loading that")
        full_year_cube = load_cube_from_picklefile(pickle_file_filepath)
    else:
        print("Pickle file doesnt exist, so creating and then saving that")

        ### Get the data filepaths
        print(f"Loading data for year {yr}")

        # Create cube list
        cubes = load_files_to_cubelist(yr, general_filename)

        # Join them into one (with error handling to deal with times which are wrong)
        try:
            full_year_cube = cubes.concatenate_cube()
            print("Concatenation successful!")
        except Exception as e:
            print(f"Initial concatenation failed: {str(e)}")

            # If initial concatenation fails, remove problematic cubes and try again
            try:
                full_year_cube = remove_problematic_cubes(cubes)
                print("Concatenation successful after removing problematic cubes!")
            except RuntimeError as e:
                print(f"Concatenation failed after removing problematic cubes: {str(e)}")               
        save_cube_as_pickle_file(full_year_cube, pickle_file_filepath)
        
        
    gauge_nums = range(0,1294)
    # Function to process each gauge
    for gauge_num in gauge_nums:
        if not gauge_num in [423, 444, 827, 888]:
                print(f"year is {yr}, gauge num is {gauge_num}")             

                base_dir = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear"
                # Create the directory if it doesnt exist
                if not os.path.isdir(base_dir):
                    os.makedirs(base_dir)

                ######################################################
                ## Check if any files are missing, across the 3 filtering options
                # If there are: code will continue to run
                # If not: code will move to next gauge
                ######################################################                
                # Create a flag to record whether we are missing any of the files we need
                missing_files = False

                # Check if we are missing any of the files, and if so, change the flag to True
                if not any(os.path.exists(f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv") for duration in [0.5, 1, 2, 3, 6, 12, 24]):
                    missing_files = True

                # If we are missing some files then get the data for the grid cell, 
                if missing_files:
                    # Find location
                    Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(tbo_vals, gauge_num, sample_cube)

                    # Extract data for the specified indices
                    start= time.time()
                    one_location_cube = full_year_cube[:, idx_2d[0], idx_2d[1]]
                    data = one_location_cube.data
                    end=time.time()
                    print(f"Time to load data is {round(end-start,2)} seconds")

                    ##### Filter cube according to different options
                    # Convert to dataframe
                    df = pd.DataFrame(data, columns=['precipitation (mm/hr)'])
                    df['times'] = one_location_cube.coord('time').units.num2date(one_location_cube.coord('time').points)
                    df['precipitation (mm)'] = df['precipitation (mm/hr)'] / 2   

                    # Search dataframe for events corresponding to durations
                    for duration in [0.5, 1, 2, 3, 6, 12, 24]:

                        filename =  f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv"
                        if not os.path.exists(filename):
                            print(f"Finding the AMAX for {duration}hr events for gauge {gauge_num} in year {yr}")
                            # Find events
                            events_v2 = search_for_valid_events(df, duration=duration, Tb0=Tb0)

                            # Save events to CSV
                            for num, event in enumerate(events_v2):
                                if len(event) > 1:
                                        event.to_csv(f"{base_dir}/{duration}hrs_{yr}_v2_part{num}.csv")
                                        if event['precipitation (mm/hr)'].isna().any():
                                            print("NANs in this event")
                        else:
                            # print(f"already exists{filename}")
                            pass   
                else:
                    pass
                    # print("already exist")



2010
Pickle file exists, so loading that
year is 2010, gauge num is 0
year is 2010, gauge num is 1
year is 2010, gauge num is 2
year is 2010, gauge num is 3
year is 2010, gauge num is 4
year is 2010, gauge num is 5
year is 2010, gauge num is 6
year is 2010, gauge num is 7
year is 2010, gauge num is 8
year is 2010, gauge num is 9
year is 2010, gauge num is 10
year is 2010, gauge num is 11
year is 2010, gauge num is 12
year is 2010, gauge num is 13
year is 2010, gauge num is 14
year is 2010, gauge num is 15
year is 2010, gauge num is 16
year is 2010, gauge num is 17
year is 2010, gauge num is 18
year is 2010, gauge num is 19
year is 2010, gauge num is 20
year is 2010, gauge num is 21
year is 2010, gauge num is 22
year is 2010, gauge num is 23
year is 2010, gauge num is 24
year is 2010, gauge num is 25
year is 2010, gauge num is 26
year is 2010, gauge num is 27
year is 2010, gauge num is 28
year is 2010, gauge num is 29
year is 2010, gauge num is 30
year is 2010, gauge num is 31
year is 2

year is 2010, gauge num is 418
year is 2010, gauge num is 419
year is 2010, gauge num is 420
year is 2010, gauge num is 421
year is 2010, gauge num is 422
year is 2010, gauge num is 424
year is 2010, gauge num is 425
year is 2010, gauge num is 426
year is 2010, gauge num is 427
year is 2010, gauge num is 428
year is 2010, gauge num is 429
year is 2010, gauge num is 430
year is 2010, gauge num is 431
year is 2010, gauge num is 432
year is 2010, gauge num is 433
year is 2010, gauge num is 434
year is 2010, gauge num is 435
year is 2010, gauge num is 436
year is 2010, gauge num is 437
year is 2010, gauge num is 438
year is 2010, gauge num is 439
year is 2010, gauge num is 440
year is 2010, gauge num is 441
year is 2010, gauge num is 442
year is 2010, gauge num is 443
year is 2010, gauge num is 445
year is 2010, gauge num is 446
year is 2010, gauge num is 447
year is 2010, gauge num is 448
year is 2010, gauge num is 449
year is 2010, gauge num is 450
year is 2010, gauge num is 451
year is 

year is 2010, gauge num is 697
year is 2010, gauge num is 698
year is 2010, gauge num is 699
year is 2010, gauge num is 700
year is 2010, gauge num is 701
year is 2010, gauge num is 702
year is 2010, gauge num is 703
year is 2010, gauge num is 704
year is 2010, gauge num is 705
year is 2010, gauge num is 706
year is 2010, gauge num is 707
year is 2010, gauge num is 708
year is 2010, gauge num is 709
year is 2010, gauge num is 710
year is 2010, gauge num is 711
year is 2010, gauge num is 712
year is 2010, gauge num is 713
year is 2010, gauge num is 714
year is 2010, gauge num is 715
year is 2010, gauge num is 716
year is 2010, gauge num is 717
year is 2010, gauge num is 718
year is 2010, gauge num is 719
year is 2010, gauge num is 720
year is 2010, gauge num is 721
year is 2010, gauge num is 722
year is 2010, gauge num is 723
year is 2010, gauge num is 724
year is 2010, gauge num is 725
year is 2010, gauge num is 726
year is 2010, gauge num is 727
year is 2010, gauge num is 728
year is 

year is 2010, gauge num is 968
year is 2010, gauge num is 969
year is 2010, gauge num is 970
year is 2010, gauge num is 971
year is 2010, gauge num is 972
year is 2010, gauge num is 973
year is 2010, gauge num is 974
year is 2010, gauge num is 975
year is 2010, gauge num is 976
year is 2010, gauge num is 977
year is 2010, gauge num is 978
year is 2010, gauge num is 979
year is 2010, gauge num is 980
year is 2010, gauge num is 981
year is 2010, gauge num is 982
year is 2010, gauge num is 983
year is 2010, gauge num is 984
year is 2010, gauge num is 985
year is 2010, gauge num is 986
year is 2010, gauge num is 987
year is 2010, gauge num is 988
year is 2010, gauge num is 989
year is 2010, gauge num is 990
year is 2010, gauge num is 991
year is 2010, gauge num is 992
year is 2010, gauge num is 993
year is 2010, gauge num is 994
year is 2010, gauge num is 995
year is 2010, gauge num is 996
year is 2010, gauge num is 997
year is 2010, gauge num is 998
year is 2010, gauge num is 999
year is 

year is 2010, gauge num is 1274
year is 2010, gauge num is 1275
year is 2010, gauge num is 1276
year is 2010, gauge num is 1277
year is 2010, gauge num is 1278
year is 2010, gauge num is 1279
year is 2010, gauge num is 1280
year is 2010, gauge num is 1281
year is 2010, gauge num is 1282
year is 2010, gauge num is 1283
year is 2010, gauge num is 1284
year is 2010, gauge num is 1285
year is 2010, gauge num is 1286
year is 2010, gauge num is 1287
year is 2010, gauge num is 1288
year is 2010, gauge num is 1289
year is 2010, gauge num is 1290
year is 2010, gauge num is 1291
year is 2010, gauge num is 1292
year is 2010, gauge num is 1293
2011
Pickle file exists, so loading that
year is 2011, gauge num is 0
year is 2011, gauge num is 1
year is 2011, gauge num is 2
year is 2011, gauge num is 3
year is 2011, gauge num is 4
year is 2011, gauge num is 5
year is 2011, gauge num is 6
year is 2011, gauge num is 7
year is 2011, gauge num is 8
year is 2011, gauge num is 9
year is 2011, gauge num is 10

...

Event doesnt contain NAN, total event precip is 38.37085723876953
Finding the AMAX for 12hr events for gauge 300 in year 2013
Event doesnt contain NAN, total event precip is 44.39580154418945
Finding the AMAX for 24hr events for gauge 300 in year 2013
Event doesnt contain NAN, total event precip is 54.6935920715332
year is 2013, gauge num is 301
(55, 340)
Time to load data is 12.63 seconds
Finding the AMAX for 0.5hr events for gauge 301 in year 2013
Event doesnt contain NAN, total event precip is 11.873255729675293
Finding the AMAX for 1hr events for gauge 301 in year 2013
Event doesnt contain NAN, total event precip is 38.98299026489258
Finding the AMAX for 2hr events for gauge 301 in year 2013
Event doesnt contain NAN, total event precip is 38.98299026489258
Finding the AMAX for 3hr events for gauge 301 in year 2013
Event doesnt contain NAN, total event precip is 38.98299026489258
Finding the AMAX for 6hr events for gauge 301 in year 2013
Event doesnt contain NAN, total event precip 

(78, 327)
Time to load data is 13.48 seconds
Finding the AMAX for 0.5hr events for gauge 310 in year 2013
Event doesnt contain NAN, total event precip is 14.694901466369629
Finding the AMAX for 1hr events for gauge 310 in year 2013
Event doesnt contain NAN, total event precip is 14.694901466369629
Finding the AMAX for 2hr events for gauge 310 in year 2013
Event doesnt contain NAN, total event precip is 37.25416564941406
Finding the AMAX for 3hr events for gauge 310 in year 2013
Event doesnt contain NAN, total event precip is 37.25416564941406
Finding the AMAX for 6hr events for gauge 310 in year 2013
Event doesnt contain NAN, total event precip is 37.25416946411133
Finding the AMAX for 12hr events for gauge 310 in year 2013
Event doesnt contain NAN, total event precip is 37.43452453613281
Finding the AMAX for 24hr events for gauge 310 in year 2013
Event doesnt contain NAN, total event precip is 37.4785041809082
year is 2013, gauge num is 311
(77, 334)
Time to load data is 14.17 seconds

Event doesnt contain NAN, total event precip is 30.96963119506836
year is 2013, gauge num is 319
(88, 335)
Time to load data is 20.84 seconds
Finding the AMAX for 0.5hr events for gauge 319 in year 2013
Event doesnt contain NAN, total event precip is 9.248617172241211
Finding the AMAX for 1hr events for gauge 319 in year 2013
Event doesnt contain NAN, total event precip is 11.655670166015625
Finding the AMAX for 2hr events for gauge 319 in year 2013
Event doesnt contain NAN, total event precip is 22.0968074798584
Finding the AMAX for 3hr events for gauge 319 in year 2013
Event doesnt contain NAN, total event precip is 29.458789825439453
Finding the AMAX for 6hr events for gauge 319 in year 2013
Event doesnt contain NAN, total event precip is 29.458799362182617
Finding the AMAX for 12hr events for gauge 319 in year 2013
Event doesnt contain NAN, total event precip is 29.498126983642578
Finding the AMAX for 24hr events for gauge 319 in year 2013
2 events
Event doesnt contain NAN, total e

(74, 347)
Time to load data is 14.94 seconds
Finding the AMAX for 0.5hr events for gauge 328 in year 2013
Event doesnt contain NAN, total event precip is 19.367271423339844
Finding the AMAX for 1hr events for gauge 328 in year 2013
Event doesnt contain NAN, total event precip is 19.367271423339844
Finding the AMAX for 2hr events for gauge 328 in year 2013
Event doesnt contain NAN, total event precip is 19.39759063720703
Finding the AMAX for 3hr events for gauge 328 in year 2013
Event doesnt contain NAN, total event precip is 25.354259490966797
Finding the AMAX for 6hr events for gauge 328 in year 2013
Event doesnt contain NAN, total event precip is 25.354263305664062
Finding the AMAX for 12hr events for gauge 328 in year 2013
Event doesnt contain NAN, total event precip is 25.370346069335938
Finding the AMAX for 24hr events for gauge 328 in year 2013
2 events
Event doesnt contain NAN, total event precip is 25.370391845703125
year is 2013, gauge num is 329
(66, 340)
Time to load data is

(75, 355)
Time to load data is 14.96 seconds
Finding the AMAX for 0.5hr events for gauge 337 in year 2013
Event doesnt contain NAN, total event precip is 22.635732650756836
Finding the AMAX for 1hr events for gauge 337 in year 2013
Event doesnt contain NAN, total event precip is 22.635732650756836
Finding the AMAX for 2hr events for gauge 337 in year 2013
Event doesnt contain NAN, total event precip is 26.062255859375
Finding the AMAX for 3hr events for gauge 337 in year 2013
Event doesnt contain NAN, total event precip is 22.63573455810547
Finding the AMAX for 6hr events for gauge 337 in year 2013
Event doesnt contain NAN, total event precip is 26.062255859375
Finding the AMAX for 12hr events for gauge 337 in year 2013
Event doesnt contain NAN, total event precip is 26.062253952026367
Finding the AMAX for 24hr events for gauge 337 in year 2013
2 events
Event doesnt contain NAN, total event precip is 26.062376022338867
year is 2013, gauge num is 338
(73, 358)
Time to load data is 13.04